In [ ]:
# to see where the notebook is placed in the system

import os
os.getcwd()

In [14]:
#to list all the files inside the directory

os.listdir()

['.ipynb_checkpoints',
 'contents',
 'NETFLIX RECOMENDATION SYSTEM.ipynb',
 'netflixData.csv']

In [11]:
#numpy is for mathematical library for computations
import numpy as np

#pandas for the excel handling it handels the two dimensional data
import pandas as pd

#sklearn is the magical library. it is the magic band of machine learning for all the supervised and unsupervised techniques
#feature extraction is inside sklearn. in feature extraction we need only text for this project
from sklearn.feature_extraction import text

#in the module pairsie there is a file called cosine_similarity-IT IS USED TO RECOMMEND
from sklearn.metrics.pairwise import cosine_similarity


In [15]:
#upload the dataset

data=pd.read_csv("netflixData.csv")

In [18]:
# read the heads to check or verify

print(data.head())

                                Show Id                          Title  \
0  cc1b6ed9-cf9e-4057-8303-34577fb54477                       (Un)Well   
1  e2ef4e91-fb25-42ab-b485-be8e3b23dedb                         #Alive   
2  b01b73b7-81f6-47a7-86d8-acb63080d525  #AnneFrank - Parallel Stories   
3  b6611af0-f53c-4a08-9ffa-9716dc57eb9c                       #blackAF   
4  7f2d4170-bab8-4d75-adc2-197f7124c070               #cats_the_mewvie   

                                         Description  \
0  This docuseries takes a deep dive into the luc...   
1  As a grisly virus rampages a city, a lone man ...   
2  Through her diary, Anne Frank's story is retol...   
3  Kenya Barris and his family navigate relations...   
4  This pawesome documentary explores how our fel...   

                      Director  \
0                          NaN   
1                       Cho Il   
2  Sabina Fedeli, Anna Migotto   
3                          NaN   
4             Michael Margolis   

             

In [20]:
#it shows which values are null in the dataset

print(data.isnull().sum())

Show Id                  0
Title                    0
Description              0
Director              2064
Genres                   0
Cast                   530
Production Country     559
Release Date             3
Rating                   4
Duration                 3
Imdb Score             608
Content Type             0
Date Added            1335
dtype: int64


In [23]:
#slicing the columns that we are going to work in

data=data[["Title","Description","Content Type","Genres"]]
print(data.head())

                           Title  \
0                       (Un)Well   
1                         #Alive   
2  #AnneFrank - Parallel Stories   
3                       #blackAF   
4               #cats_the_mewvie   

                                         Description Content Type  \
0  This docuseries takes a deep dive into the luc...      TV Show   
1  As a grisly virus rampages a city, a lone man ...        Movie   
2  Through her diary, Anne Frank's story is retol...        Movie   
3  Kenya Barris and his family navigate relations...      TV Show   
4  This pawesome documentary explores how our fel...        Movie   

                                           Genres  
0                                      Reality TV  
1  Horror Movies, International Movies, Thrillers  
2             Documentaries, International Movies  
3                                     TV Comedies  
4             Documentaries, International Movies  


In [24]:
# null values are cleaned from the data for making the dataset more lookable
data=data.dropna()

In [27]:
#print the data works only in notebooks
data

,Title,Description,Content Type,Genres
0,(Un)Well,This docuseries takes a deep dive into the luc...,TV Show,Reality TV
1,#Alive,"As a grisly virus rampages a city, a lone man ...",Movie,"Horror Movies, International Movies, Thrillers"
2,#AnneFrank - Parallel Stories,"Through her diary, Anne Frank's story is retol...",Movie,"Documentaries, International Movies"
3,#blackAF,Kenya Barris and his family navigate relations...,TV Show,TV Comedies
4,#cats_the_mewvie,This pawesome documentary explores how our fel...,Movie,"Documentaries, International Movies"
...,...,...,...,...
5962,الف مبروك,"On his wedding day, an arrogant, greedy accoun...",Movie,"Comedies, Dramas, International Movies"
5963,دفعة القاهرة,A group of women leaves Kuwait to attend unive...,TV Show,"International TV Shows, TV Dramas"
5964,海的儿子,"Two brothers start a new life in Singapore, wh...",TV Show,"International TV Shows, TV Dramas"
5965,반드시 잡는다,After people in his town start turning up dead...,Movie,"Dramas, International Movies, Thrillers"


In [32]:
#nlp library--removes the garbage words from a sentence
import nltk

#regular expression
import re

#package of stopwords is taken as an extension tool 
#download the extension
nltk.download('stopwords')

#it is a method in the nltk that is snowball which supprts all language in which we need english
#create stemmer method
stemmer=nltk.SnowballStemmer("english")

#load stopwords
from nltk.corpus import stopwords

import string
stopword=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
print(data.Genres[20:100])

20              Crime TV Shows, TV Dramas, TV Mysteries
21                           Crime TV Shows, Docuseries
22                                        Documentaries
23                                        Documentaries
24                 Comedies, Dramas, Independent Movies
                            ...                        
95    Children & Family Movies, Dramas, Romantic Movies
96    Children & Family Movies, Dramas, Romantic Movies
97                                               Movies
98    Children & Family Movies, Comedies, Music & Mu...
99                  Horror Movies, International Movies
Name: Genres, Length: 80, dtype: object


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
feature=data['Genres'].tolist()

#create an instance of TF -IDF vectorizer
tfidf = TfidfVectorizer(stop_words="english")

#fit and transform the vectorizer on our corpus
tfidf_matrix = tfidf.fit_transform(feature)

#compute the cosine similarity matrix
similarity = cosine_similarity(tfidf_matrix)

indices = pd.Series(data.index,index=data['Title']).drop_duplicates()

In [57]:
def netflix_recommendation(title):
    try:
        index = indices[title]
        similarity_scores = list(enumerate(similarity[index]))
        similarity_scores = sorted(similarity_scores,key=lambda x:x[1],reverse=True)
        similarity_scores = similarity_scores[0:10]
        movieindices = [i[0] for i in similarity_scores]
        return data['Title'].iloc[movieindices]
    except:
        print(title,"is not in the database")

In [52]:
netflix_recommendation("Aaviri")

1                           #Alive
178                         Aaviri
360                   Andhaghaaram
361                    Andhakaaram
398                        Apostle
1759     Game Over (Hindi Version)
1760     Game Over (Tamil Version)
1761    Game Over (Telugu Version)
1801                     Ghost Lab
1804                 Ghost Stories
Name: Title, dtype: object

In [53]:
netflix_recommendation("#Alive")

1                           #Alive
178                         Aaviri
360                   Andhaghaaram
361                    Andhakaaram
398                        Apostle
1759     Game Over (Hindi Version)
1760     Game Over (Tamil Version)
1761    Game Over (Telugu Version)
1801                     Ghost Lab
1804                 Ghost Stories
Name: Title, dtype: object

In [58]:
netflix_recommendation("Tiger 3")

Tiger 3 is not in the database
